## Подключение библиотек

In [ ]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None

## E-Greedy baseline

In [ ]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        # choose random banner
        return history.index[randint.rvs(0, n)]

    # the number of clicks that your ad receives divided by the number of times your ad is shown
    ctr = history['clicks'] / (history['impressions'] + 1)
    n = np.argmax(ctr)
    return history.index[n]


policy = partial(eps_greedy, eps=0.06)

## Тестирование бейзлайна

In [ ]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

In [ ]:
# baseline regret
output['regret'], output['regret'] / output['rounds'], output['total_banners']

In [ ]:
output['history']

## UCB policy
Реализуем UCB бандита

In [ ]:
def ucb(history: pd.DataFrame, exploration_coef: float):
    # добавляем +1, чтобы впоследствии не возникло деления на 0
    impressions = history['impressions'] + 1
    # кол-во кликов на рекламный банер делить на суммарное число показов банера 
    ctr_exploitation = history['clicks'] / impressions

    # exploration часть
    total_impressions = np.sum(impressions)
    exploration = np.sqrt(2 * np.log(total_impressions) / impressions)

    # домножаем exploration часть на коэффициент, чтобы достичь баланса между exploitation и exploration 
    # (т.е между уже известной и еще неисследованной частью)
    u = ctr_exploitation + exploration_coef * exploration

    # выбираем argmax u по всем баннерам
    best_banner_id = np.argmax(u)
    return history.index[best_banner_id]

## Подбор гиперпараметра: коэффициент при exploration части

In [ ]:
seed = 18475

def test_ucb_policy(exploration_coef: float, process_printing=False):
    np.random.seed(seed=seed)
    ucb_policy = partial(ucb, exploration_coef=exploration_coef)
    output = simulation(ucb_policy, n=200000, seed=seed, process_printing=process_printing)
    print(f"coef: {exploration_coef}, regret: {output['regret']:.4f}, "
          f"average regret: {(output['regret'] / output['rounds']):.4f}, banners: {output['total_banners']}")

Коэффициент при exploration слагаемом не должен быть большим, тк все же в основном нам важно учесть уже полученные данные exploitation части, и мы хотим добавить небольшое рассмотрение еще недостаточно изученных баннеров из exploration.

In [ ]:
exploration_coefs = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.15, 0.2, 0.3, 0.5, 0.9, 1]

for coef in exploration_coefs:
    test_ucb_policy(coef)

Видим, что наименьшее значение регрета достигается при coef = 0.08. Для более хорошего результата можем проверить значения коэффициента в окрестностях этой точки:

In [ ]:
exploration_coefs = [0.065, 0.07, 0.075, 0.08, 0.085, 0.09]

for coef in exploration_coefs:
    test_ucb_policy(coef)

Финальное тестирование лучшего коэффициента coef = 0.07.

In [ ]:
best_exploration_coef = 0.07
test_ucb_policy(best_exploration_coef, process_printing=True)

Регрет значительно меньше, чем у бейзлайна, баланс между exploitation и exploration подобран. Успех